In [46]:
import pandas as pd
from datetime import datetime
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,RidgeCV,Ridge,Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFE



In [47]:
#'### Read the data. Filling up missing values with 0
df=pd.read_csv("weather_data/ikermit.csv.gz",compression='gzip')
df["date"] = [datetime.fromisoformat(dt[:-1]) for dt in df.date]
feature_cols = ['WindMax', 'WindGustAlwaysPast1h', 'WindMin', 'WindMinPast1h', 'TempDew', 'Pressure', 'PressureAtSea', 'WindDirPast1h', 'TempDry', 'Weather', 'TempMeanPast1h', 'TempMaxPast1h', 'TempMinPast1h', 'Humidity', 'HumidityPast1h', 'Visibility']

print(f"Using {len(feature_cols)} features: {feature_cols}")
target_col=["WindSpeed"]

Using 16 features: ['WindMax', 'WindGustAlwaysPast1h', 'WindMin', 'WindMinPast1h', 'TempDew', 'Pressure', 'PressureAtSea', 'WindDirPast1h', 'TempDry', 'Weather', 'TempMeanPast1h', 'TempMaxPast1h', 'TempMinPast1h', 'Humidity', 'HumidityPast1h', 'Visibility']


In [48]:
#data splitting again

In [131]:
beg_date = datetime(2021,8,1)
end_date = datetime(2021,12,1)
df = df[(df["date"] > beg_date ) & (df["date"] < end_date)]
print(f"Shape after filtering year 2020: {df.shape}")
df.fillna(0,inplace=True)
df.sort_values(by=["date"],inplace=True)

df_train_full, df_test = train_test_split(df, test_size=0.2,random_state=4,shuffle=False)
df_train, df_val = train_test_split(df_train_full, test_size=0.25,random_state=42, shuffle=False)

Shape after filtering year 2020: (17536, 23)


In [132]:
#This to be used at the end!!!
this_date = "2021-11-07"
df_test = df_test[df_test["date"] == this_date]
X_test = df_test[feature_cols]
y_test = df_test[target_col].values

In [133]:
type(df_test)

pandas.core.frame.DataFrame

In [134]:
type(df_train)

pandas.core.frame.DataFrame

### Model setup, using the training data

In [135]:
# splitting and Data normalization
X_train = df_train[feature_cols]
y_train = df_train[target_col].values

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


In [137]:
type(X_train)

numpy.ndarray

In [138]:
# Load the model
import pickle
with open("rnn_model.bin","rb") as f:
    model =pickle.load(f)

In [104]:
model

In [139]:
X_train.shape

(10521, 16)

In [140]:
def split_sequence2(sequence1, sequence2, n_steps):
    X, y = list(), list()
    if len(sequence1) != len(sequence2):
        raise RuntimeError
    for i in range(len(sequence1)):
    # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence1)-1:
            break
        # gather input and output parts of the pattern
        #seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        seq_x = sequence1[i:end_ix]
        seq_y = sequence2[i:end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
#now trying to test model...

In [156]:
X2_train,y2 = split_sequence2(X_train,y_train,16)

In [157]:
X2_train.shape

(10505, 16, 16)

In [160]:
X_test

array([[ 0.99040064,  3.66404868, -0.06337733, -0.08230222, -0.91032148,
        -1.32717965, -1.32687763,  2.03170462, -1.68099128,  2.13284492,
         1.20038893,  1.18847873,  1.21080218,  1.12485789,  2.67226485,
        -1.8707884 ]])

In [158]:
X2_test = np.array([X_test])

In [161]:
X2_test.shape

(1, 1, 16)

In [162]:
# testing = np.array(X_test)
# testing = np.reshape(testing,(testing.shape[1],testing.shape[0],1))

In [163]:
type(X_test)

numpy.ndarray

In [164]:
X_train.shape

(10521, 16)

In [167]:
y_pred = model.predict(X2_test)


ValueError: in user code:

    File "/media/cap/7fed51bd-a88e-4971-9656-d617655b6312/data/sources/conda/miniconda3/envs/py39/lib/python3.9/site-packages/keras/engine/training.py", line 1621, in predict_function  *
        return step_function(self, iterator)
    File "/media/cap/7fed51bd-a88e-4971-9656-d617655b6312/data/sources/conda/miniconda3/envs/py39/lib/python3.9/site-packages/keras/engine/training.py", line 1611, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/media/cap/7fed51bd-a88e-4971-9656-d617655b6312/data/sources/conda/miniconda3/envs/py39/lib/python3.9/site-packages/keras/engine/training.py", line 1604, in run_step  **
        outputs = model.predict_step(data)
    File "/media/cap/7fed51bd-a88e-4971-9656-d617655b6312/data/sources/conda/miniconda3/envs/py39/lib/python3.9/site-packages/keras/engine/training.py", line 1572, in predict_step
        return self(x, training=False)
    File "/media/cap/7fed51bd-a88e-4971-96